# RNN with LSTM cells on IMDB and pre-trained GloVe word vectors







### Imports

In [1]:
import torch
import torchtext 
import random
import spacy
from torch import nn
import torch.nn.functional as F

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

### Settings and Dataset

In [2]:
# Device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


# Hyperparameters
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-4
BATCH_SIZE = 128
NUM_EPOCHS = 10
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

In [3]:
TEXT = torchtext.legacy.data.Field(tokenize="basic_english",
                  include_lengths=True) 
LABEL = torchtext.legacy.data.LabelField(dtype=torch.float)


train_data, test_data = torchtext.legacy.datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(RANDOM_SEED),
                                          split_ratio=0.8)


print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 20000
Num Valid: 5000
Num Test: 25000


In [4]:
#TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE, vectors='glove.6B.100d', unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)


print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [5]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    device=DEVICE)

In [6]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    

print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Train
Text matrix size: torch.Size([132, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([61, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([42, 128])
Target vector size: torch.Size([128])


### Model

In [7]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_length):
        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length.cpu())
        
        #[sentence len, batch size, embedding size] => 
        #  output: [sentence len, batch size, hidden size]
        #  hidden: [1, batch size, hidden size]
        packed_output, (hidden, cell) = self.rnn(packed)
        return self.fc(hidden.squeeze(0)).view(-1)

In [8]:
INPUT_DIM = len(TEXT.vocab)
torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

### Training

In [9]:
def compute_binary_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.text
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float()/num_examples * 100


for epoch in range(NUM_EPOCHS):
    model.train()
    
    for batch_idx, batch_data in enumerate(train_loader):
        
      text, text_lengths = batch_data.text
        
      # FORWARD AND BACK PROP
      logits = model(text, text_lengths)
      cost = F.binary_cross_entropy_with_logits(logits, batch_data.label)
      optimizer.zero_grad()
        
      cost.backward()
        
      # UPDATE MODEL PARAMETERS
      optimizer.step()
        
      # Loggingcell
      if not batch_idx % 50:
        print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d | Train: %.3f%% ' %(
              epoch+1, NUM_EPOCHS, 
              compute_binary_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 0000/0157 | Cost: 0.6907
Epoch: 001/010 | Batch 0050/0157 | Cost: 0.6802
Epoch: 001/010 | Batch 0100/0157 | Cost: 0.6555
Epoch: 001/010 | Batch 0150/0157 | Cost: 0.6369
Epoch: 001/010 | Train: 61.020% 
Epoch: 002/010 | Batch 0000/0157 | Cost: 0.6738
Epoch: 002/010 | Batch 0050/0157 | Cost: 0.5763
Epoch: 002/010 | Batch 0100/0157 | Cost: 0.6044
Epoch: 002/010 | Batch 0150/0157 | Cost: 0.6212
Epoch: 002/010 | Train: 71.835% 
Epoch: 003/010 | Batch 0000/0157 | Cost: 0.5559
Epoch: 003/010 | Batch 0050/0157 | Cost: 0.5782
Epoch: 003/010 | Batch 0100/0157 | Cost: 0.5587
Epoch: 003/010 | Batch 0150/0157 | Cost: 0.5305
Epoch: 003/010 | Train: 75.810% 
Epoch: 004/010 | Batch 0000/0157 | Cost: 0.4819
Epoch: 004/010 | Batch 0050/0157 | Cost: 0.4011
Epoch: 004/010 | Batch 0100/0157 | Cost: 0.5150
Epoch: 004/010 | Batch 0150/0157 | Cost: 0.5585
Epoch: 004/010 | Train: 77.305% 
Epoch: 005/010 | Batch 0000/0157 | Cost: 0.4203
Epoch: 005/010 | Batch 0050/0157 | Cost: 0.4684
Epoc

### Evaluation

In [10]:
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [11]:
print('Probability positive:')
predict_sentiment(model, "I really love this movie. This movie is so great!")

Probability positive:


0.7735075354576111